<a href="https://colab.research.google.com/github/RyanTokManMokMTM/NLP_Training2/blob/main/web_crawler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#TODO
'''
baseURL:https://movies.yahoo.com.tw/
playingNOW:/movie_intheaters.html
Upcoming:movie_comingsoon.html
ThisWeek:/movie_thisweek.html

Accessing the page ?page=x
'''

'\nbaseURL:https://movies.yahoo.com.tw/\nplayingNOW:/movie_intheaters.html\nUpcoming:movie_comingsoon.html\nThisWeek:/movie_thisweek.html\n\nAccessing the page ?page=x\n'

In [2]:
!pip install pandas #install pandas

In [3]:
'''
Find The rule of first Movie
ForCurrentPage: 
All movie is inside  class = "release_list"
Get The URL of the introl button: class="release_btn color_btnbox" first a tag[href']


1.ForEach movies, need to get its detail page URL
2....
'''

'\nFind The rule of first Movie\nForCurrentPage: \nAll movie is inside  class = "release_list"\nGet The URL of the introl button: class="release_btn color_btnbox" first a tag[href\']\n\n\n1.ForEach movies, need to get its detail page URL\n2....\n'

In [4]:
import requests
from bs4 import BeautifulSoup
HEADER = {
       'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'
}


type_list = {
    "playing":"/movie_intheaters.html",
    "upcoming":"/movie_comingsoon.html",
    "thisweek":"/movie_thisweek.html"
}

baseURL = "https://movies.yahoo.com.tw"+ type_list["playing"]
print(baseURL)

#now sending the request
movieReq = requests.get(baseURL,headers=HEADER)
currentPage = BeautifulSoup(movieReq.text)

#get all movie data from `
movieList = currentPage.find("div",class_="release_box").find("ul",class_="release_list").find("a",class_="btn_s_introduction")['href']
print(movieList)

#now we need to get all data for the movie
#now we get the name and its link for tesing
movieData = currentPage.findAll("div",class_="release_info")
for item in movieData:
    #get movie name
    print(item.find("div",class_="release_movie_name").find("a",class_="gabtn").text.strip())

    #get movie detail URL
    print(item.find("div",class_="release_btn color_btnbox").find("a",class_="btn_s_introduction")["href"])


https://movies.yahoo.com.tw/movie_intheaters.html
https://movies.yahoo.com.tw/movieinfo_main/%E8%B7%AF%E5%8D%A1%E7%9A%84%E5%A4%8F%E5%A4%A9-luca-11268
路卡的夏天
https://movies.yahoo.com.tw/movieinfo_main/%E8%B7%AF%E5%8D%A1%E7%9A%84%E5%A4%8F%E5%A4%A9-luca-11268
薄荷糖
https://movies.yahoo.com.tw/movieinfo_main/%E8%96%84%E8%8D%B7%E7%B3%96-peppermint-candy-11368
人聲
https://movies.yahoo.com.tw/movieinfo_main/%E4%BA%BA%E8%81%B2-the-human-voice-11400
雙面薇若妮卡 經典數位修復
https://movies.yahoo.com.tw/movieinfo_main/%E9%9B%99%E9%9D%A2%E8%96%87%E8%8B%A5%E5%A6%AE%E5%8D%A1-%E7%B6%93%E5%85%B8%E6%95%B8%E4%BD%8D%E4%BF%AE%E5%BE%A9-the-double-life-of-veronique-11425
合法伴侶
https://movies.yahoo.com.tw/movieinfo_main/%E5%90%88%E6%B3%95%E4%BC%B4%E4%BE%B6-special-couple-11441
甲子園：夢想競技場
https://movies.yahoo.com.tw/movieinfo_main/%E7%94%B2%E5%AD%90%E5%9C%92-%E5%A4%A2%E6%83%B3%E7%AB%B6%E6%8A%80%E5%A0%B4-koshien-japans-field-of-dreams-11449
青春：第50屆日本全國高中棒球錦標賽
https://movies.yahoo.com.tw/movieinfo_main/%E9%9D%92%E6%98%A5-%E7%AC

In [ ]:
'''
Find the rule of detail movie page
Data is needed: movieName, Genra(order is required) and Introl
Basic : class = movie_intro_info_r
Genre : level_name->a->text
Introl: class = gray_infobox_inner -> story -> story ->p
'''

In [18]:
#for now just getting the page data
#Ignore other related movie for now
#later fix existing movie

# for page in range(1,8):
#   webURL = baseURL + "?page=%s"%page
#   #send to request to 

#now we need to get the info we want
tempURL = "https://movies.yahoo.com.tw/movieinfo_main/%E8%B7%AF%E5%8D%A1%E7%9A%84%E5%A4%8F%E5%A4%A9-luca-11268"
reqDetail = requests.get(tempURL,headers=HEADER)
detailSoup = BeautifulSoup(reqDetail.text)
# print(detailSoup)

#getting the basic 
# detailSoup.find("div",class_="movie_intro_info_r")
info = detailSoup.find("div",class_="movie_intro_info_r")
print("MovieName",info.find("h1").text.strip())
genreList = info.findAll("div",class_="level_name")

#getting movie genre
for genre in genreList:
  print(genre.find("a",class_="gabtn").text.strip())

#getting movie intro
intro = detailSoup.find("div",class_="gray_infobox_inner")
introData = intro.find("span",id="story").text.strip()
print(introData)


MovieName 路卡的夏天
動畫
冒險
皮克斯全新原創《路卡的夏天》，故事背景發生在美麗的義大利濱海小鎮里維埃拉，一位名叫路卡的海怪男孩的難忘夏天，他與新摯友們在夏天經歷的無數冒險，包含他們在島上參加的競賽，但最刺激的不是競賽本身，而是隱藏他們呼之欲出的海怪身份…
 
《路卡的夏天》是一個成長的故事，海怪男孩是曾經的你我，好奇心驅使你走出舒適圈，遇見真誠的友誼，學習鼓起勇氣由衷生活，完成自以為不可能的夢想…
